In [396]:
import networkx as nx
import pandas as pd

df_edges = pd.read_csv('../data/edgelist.csv')
G = nx.from_pandas_edgelist(df_edges)

In [444]:
G = nx.read_edgelist("../data/test_louvain.txt", nodetype = int)

In [445]:
from community import community_louvain
import numpy as np

# Compute Louvain method
partitions = community_louvain.best_partition(G)
unique_coms = np.unique(list(partitions.values()))

print(partitions)

{0: 1, 2: 1, 4: 1, 3: 1, 5: 1, 1: 1, 7: 1, 6: 1, 10: 0, 11: 0, 8: 0, 9: 0, 14: 0, 15: 0, 12: 0, 13: 0}


In [446]:
import networkx.algorithms.community as nx_comm

nx_comm.louvain_communities(G, seed=123)

[{0, 1, 2, 3, 4, 5, 6, 7}, {8, 9, 10, 11, 12, 13, 14, 15}]

In [448]:
from networkx.utils import py_random_state

def gain(G, ni, nodes, hypernode=False):
   m = G.number_of_edges()
   
   dn = lambda n: G.degree[n]
   if hypernode:
      dn = lambda n: G[n][n]["weight"] if G.has_edge(n, n) else 0

   di = dn(ni)
   dj = sum(dn(nj) for nj in nodes if ni != nj)
   
   dij = 2 * sum(G[ni][nj]["weight"] for nj in nodes if G.has_edge(ni, nj) and ni != nj)


   return 0.5 * (dij - di * dj / m) / m

@py_random_state("seed")
def get_partition(G, hypernode=False, seed=None):
   partition = [{u} for u in G.nodes()]
   node_com = {u: i for i, u in enumerate(G.nodes())}
   rand_nodes = list(G.nodes)
   seed.shuffle(rand_nodes)
   loop = True
   nbrs = {u: {v for v in G[u] if v != u} for u in G}

   while loop:
      
      loop = False
      
      for node in rand_nodes:

         neighbors_com = set(node_com[j] for j in nbrs[node])
         best_com = max_dQ = -1
         
         for com in neighbors_com:
            dQ = gain(G, node, partition[com], hypernode=hypernode)

            if dQ > max_dQ:
               max_dQ = dQ
               best_com = com

         if max_dQ > 0 and best_com != node_com[node]:
            partition[node_com[node]].remove(node)
            partition[best_com].add(node)
            node_com[node] = best_com
            loop = True
            
   return [c for c in partition if c]

def create_graph(G, partition):
   G2 = nx.Graph()
   node_com = {}
   
   for i, com in enumerate(partition):
      for node in com: 
         node_com[node] = i
      G2.add_node(i) 

   for node1, node2 in G.edges():
      com1, com2 = node_com[node1], node_com[node2]
      weight = 1 if com1 != com2 else 2
      if G2.has_edge(com1, com2):
         G2[com1][com2]["weight"] += weight
      else:
         G2.add_edge(com1, com2, weight=weight)

   return G2

def complete_partition(part, partitions):
   if not partitions:
      return part, False

   part_complete = []
   for com in part:
      tmp = set()
      for node in com:
         tmp |= partitions[-1][node] 
      part_complete.append(tmp)

   return part_complete, len(part) == len(partitions[-1])

def louvain_partitions(graph):
   G = nx.Graph()
   G.add_nodes_from(graph)
   G.add_weighted_edges_from(graph.edges(data="weight", default=1))

   partitions = []

   while True:
      part = get_partition(G)
      partition, convergence = complete_partition(part, partitions)
 
      if convergence:
         break

      partitions.append(partition)
      G = create_graph(G, partition)
   
   return partitions

In [449]:
louvain_partitions(G)

[[{0, 1, 2, 4, 5}, {3, 6, 7}, {8, 9, 10, 12, 14, 15}, {11, 13}],
 [{0, 1, 2, 3, 4, 5, 6, 7}, {8, 9, 10, 11, 12, 13, 14, 15}]]